# Demystifying Multiple Inheritance in Python

## Example of Multiple Inheritance

In [ ]:
from collections import UserString, Counter

class StringCounter(UserString, Counter):

    def __init__(self, initial):
        UserString.__init__(self, initial)
        Counter.__init__(self, initial)

sc = StringCounter('succeeded')

# You can use UserString (str) methods
print(f'{sc = }')
print(f'{sc[0] = }')
print(f'{len(sc) = }')
print(f'{sc.upper() = }')

# And also Counter (dict) methods
print(f'{sc.items() = }')
print(f'{sc.keys() = }')
print(f'{sc.most_common() = }')

## How Python Handles the Diamond Problem

In [ ]:
class Base(object):
    def method(self):
        print('Method in Base')

class Left(Base):
    def method(self):
        print('Method in Left')

class Right(Base):
    def method(self):
        print('Method in Right')

class Derived(Left, Right):
    pass

d = Derived()
d.method()

# from pprint import pp
# pp(Derived.__mro__)

## MRO Example 1

In [ ]:
class A(object): pass
class B(A): pass
class C(A): pass
class D(B, C): pass

from pprint import pp
pp(D.__mro__)

## MRO Example 2

In [ ]:
class A(object): pass
class B(A): pass
class C(A): pass
# The following line produces an error
class D(B, A, C): pass  # type: ignore

from pprint import pp
pp(D.__mro__)

## A `super()` Example

In [ ]:
class A(object):
    def method(self):
        return 'A'

class B(A):
    def method(self):
        return 'B' + super().method()

class C(A):
    def method(self):
        return 'C' + super().method()

class D(B, C):
    def method(self):
        return 'D' + super().method()

class E(B, C):
    def method(self):
        return 'E' + super().method()

class F(D, E):
    def method(self):
        # These three are equivalent:
        return 'F' + super().method()
        # return 'F' + super(F, self).method()
        # return 'F' + D.method(self)

        # These two are equivalente:
        # return 'F' + A.method(self)
        # return 'F' + super(C, self).method()

from pprint import pp
pp(F.__mro__)

f = F()
f.method()

## Defining Your Own MRO

In [ ]:
class A(object):
    def method(self):
        if hasattr(super(), 'method'):
            return 'A' + super().method()  # type: ignore
        else:
            return 'A'

class B(A):
    def method(self):
        if hasattr(super(), 'method'):
            return 'B' + super().method()
        else:
            return 'B'

class MyMRO(type):
    def mro(cls):
        return [cls, A, B, object]

class C(B, metaclass=MyMRO):
    def method(self):
        if hasattr(super(), 'method'):
            return 'C' + super().method()
        else:
            return 'C'

from pprint import pp
pp(C.__mro__)

c = C()
c.method()

## Mixin Example

In [ ]:
import json

# A mixin for JSON serialization
class JsonMixin:
    def to_json(self):
        return json.dumps(self.__dict__)

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

# A class that uses the mixin
class Employee(JsonMixin, Person):
    def __init__(self, name, age, position):
        super().__init__(name, age)
        self.position = position

employee = Employee('Guido van Rossum', 68, 'Dutch programmer')
print(employee.to_json())

## Favor Composition over Inheritance

### Inheritance Example

In [ ]:
from collections import UserList

class StackInheritance(UserList):
    def push(self, item):
        self.append(item)

    def peek(self):
        return self[-1]

    def __len__(self):
        return super().__len__()

s = StackInheritance()
s.push('a')
s.push('b')
s.push('c')
print(f'{s = }')
print(f'{s.pop() = }')
print(f'{s.peek() = }')
print(f'{len(s) = }')

# You can still use all the UserList methods!
print(f'{s[0] = }')
s.insert(0, 'd')
print(f'{s = }')

### Composition Example

In [ ]:
from collections import UserList
from typing import Any

class StackComposition:
    def __init__(self):
        self.__list = UserList()

    def push(self, item):
        self.__list.append(item)

    def pop(self):
        return self.__list.pop()

    def peek(self):
        return self.__list[-1]

    def __len__(self):
        return len(self.__list)

    def __repr__(self):
        return repr(self.__list)

s = StackComposition()
s.push('a')
s.push('b')
s.push('c')
print(f'{s = }')
print(f'{s.pop() = }')
print(f'{s.peek() = }')
print(f'{len(s) = }')

# You can't use directly the UserList methods!
# print(f'{s[0] = }')
# s.insert(0, 'd')
# print(f'{s = }')

## ISP Example

In [ ]:
from typing import Protocol

class InputProtocol(Protocol):
    def read(self) -> str:
        ...

class OutputProtocol(Protocol):
    def write(self, data: str) -> None:
        ...

class ConsoleOutput(OutputProtocol):
    def write(self, data: str) -> None:
        print(f'Output: {data}')

class ConsoleInput(InputProtocol):
    def read(self) -> str:
        return input('Input: ')

class ConsoleInputOutput(ConsoleInput, ConsoleOutput):
    ...

def write_something(outp: OutputProtocol, data: str) -> None:
    print('Writing something...')
    outp.write(data)

def read_something(inp: InputProtocol) -> None:
    print('Reading something...')
    print(f'Data read: {inp.read()}')

In [ ]:
console_out = ConsoleOutput()
write_something(console_out, 'some data')
# read_something(console_out)  #  This line produces a static type checker error

In [ ]:
console_in = ConsoleInput()
# write_something(console_in, 'some data')  #  This line produces a static type checker error
read_something(console_in)

In [ ]:
console_in_out = ConsoleInputOutput()
write_something(console_in_out, 'some data')
read_something(console_in_out)